<a href="https://colab.research.google.com/github/alansenairj/chatbot_watson/blob/main/watson_api_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# bibliotecas auxiliares e importações de suporte

# biblioteca para executar API flask no colab
!pip install flask-ngrok

# biblioteca para entrar dados mascarados

from getpass import getpass

# configuração de usuário e senha do email smtp

u = getpass('User: ')
g = getpass('Password: ')

#módulos necessários do flask

from flask import Flask, request, jsonify, Response

#módulos para crawler das infos
import requests
from bs4 import BeautifulSoup

#módulos par aenvio de e-mail
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

#criar app flask

app = Flask(__name__, static_url_path= '')

#apenas para executar no collab
from flask_ngrok import run_with_ngrok
run_with_ngrok(app)

#mapeamento das URLs dos cursos para crawler
curso_urls = {
    
    "IA" : "https://www.impacta.edu.br/mba/artificial-intelligence", 
    'Big Data': 'https://www.impacta.edu.br/mba/business-intelligence-e-analytics'
}

#GET raiz para teste
@app.route('/')
def root():
  return "OK"

#endpoint do webhook - POST
@app.route('/watsonwebhook', methods=['POST'])
def watsonwebhook(): 
  # retornar dados enviados em formato JSON
  req = request.get_json()
  print(req)
  #return Response(status=200)
  
  #tipo de task enviada: info ou email
  task = req['task']
  if task =='info': 
    #nome do curso enviado
    curso = req['curso']

    #crawler para pegar informações
    r = requests.get(curso_urls[curso]).text
    info = BeatifulSoup(r, 'html.parser')(class_='backgroud-base-site')[0].text

    #retomar informações no campo info, no formato JSON

    return jsonify({'info': info})
  elif task == 'email': 
    try: 

    # configuração e envio de email

      message = MIMEMultipart()
      message['From'] = u
      message['To'] = req['email']
      message['Subject'] = 'Confirmação de pré-matrícula'
      texto_mensagem = f"Olá! Sua pré-matrícula no curso de {req['curso']} para o {req['semestre']} semestre está confirmada"
      message.attach(MIMEText(texto_mensagem, 'plain'))

      session = smtplib.SMTP('smtp.gmail.com', 587)
      session.starttls()
      session.login(u, g)
      text = message.as_string()
      session.sendmail(u, req['email'], text)
      session.quit()
    # retorno de sucesso no formato JSON
      return jsonify({'status': 'ok'})
    except Exception as ex: 
      print (ex)
     #retorno de erro noi formato JSON
      return jsonify({'status': 'erro'})

#executar app flask

if __name__ == '__main__': 
  app.run()






User: ··········
Password: ··········
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bcb98c09b39c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


[2021-07-23 20:21:03,805] ERROR in app: Exception on /watsonwebhook [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-1-bcdfb62d0e86>", line 63, in watsonwebhook
    r = requests.get(curso_urls[c

{'task': 'info', 'curso': None}
